Un champion du monde Pokémon (oui, cela existe) a entendu parler de tes connaissances en Machine Learning et demande ton expertise.

Le champion a l'habitude d'utiliser les Pokemon suivants : Mewtwo, Lugia, Rayquaza, Giratina, Dialga, et Palkia.

 Mais pour sa prochaine compétition, il est interdit d'utiliser un Pokémon légendaire. 
 
 Un pokemon est soit légendaire, soit non-légendaire. La dernière colonne du dataset indique True ou False suivant les cas.

 
Le champion devra donc remplacer ses Pokemon légendaires préférés par d'autres Pokémons non-légendaires, avec des caractéristiques similaires. 

C'est là que tu interviens !

Pour l'aider, suis les étapes suivantes :

Importe le dataset pokemon.csv. Chaque ligne représente un pokemon, avec ses différentes caractéristiques (attaque, défense, vitesse, etc...), ainsi que la colonne "légendaire".

Entraine l'algorithme NN en utilisant l'ensemble des colonnes numériques en variables explicatives X.


Les valeurs nulles (NaN) posent problème à l'ensemble des algorithmes. Ici, pour simplifier, tu peux supprimer la colonne concernée avant d'entrainer ton modèle.


Pas besoin de traintestsplit dans ce cas précis : en effet, sinon tu chercheras les voisins sur un sous-échantillon. Alors qu'il y a peut-être des voisins plus proche.


Pas besoin de predict ou de score non plus : nous calculons uniquement des distances.


Utilise la fonction kneighbors() pour trouver le ou les Pokémons le(s) plus proche(s) de chacun des Pokémon légendaires que le champion a l'habitude d'utiliser.


Fais des recommandations de Pokémon à utiliser par le champion pour sa prochaine compétition, en lui priorisant par distance. 

Evidemment, ta recommandation ne doit pas indiquer un autre pokemon légendaire, même si ce dernier est très proche...

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

# Charger le jeu de données
df = pd.read_csv(
    'https://raw.githubusercontent.com/murpi/wilddata/master/pokemon.csv', sep=",")

In [3]:
df.head(2)

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False


In [4]:
df.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk',
       'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [5]:
df_non_legendaire = df[df['Legendary'] == False]['Name'].unique()
print(df_non_legendaire)

['Bulbasaur' 'Ivysaur' 'Venusaur' 'Mega Venusaur' 'Charmander'
 'Charmeleon' 'Charizard' 'Mega Charizard X' 'Mega Charizard Y' 'Squirtle'
 'Wartortle' 'Blastoise' 'Mega Blastoise' 'Caterpie' 'Metapod'
 'Butterfree' 'Weedle' 'Kakuna' 'Beedrill' 'Mega Beedrill' 'Pidgey'
 'Pidgeotto' 'Pidgeot' 'Mega Pidgeot' 'Rattata' 'Raticate' 'Spearow'
 'Fearow' 'Ekans' 'Arbok' 'Pikachu' 'Raichu' 'Sandshrew' 'Sandslash'
 'Nidoran♀' 'Nidorina' 'Nidoqueen' 'Nidoran♂' 'Nidorino' 'Nidoking'
 'Clefairy' 'Clefable' 'Vulpix' 'Ninetales' 'Jigglypuff' 'Wigglytuff'
 'Zubat' 'Golbat' 'Oddish' 'Gloom' 'Vileplume' 'Paras' 'Parasect'
 'Venonat' 'Venomoth' 'Diglett' 'Dugtrio' 'Meowth' 'Persian' 'Psyduck'
 'Golduck' 'Mankey' 'Mankeyduck' 'Growlithe' 'Arcanine' 'Poliwag'
 'Poliwhirl' 'Poliwrath' 'Abra' 'Kadabra' 'Alakazam' 'Mega Alakazam'
 'Machop' 'Machoke' 'Machamp' 'Bellsprout' 'Weepinbell' 'Victreebel'
 'Tentacool' 'Tentacruel' 'Geodude' 'Graveler' 'Golem' 'Ponyta' 'Rapidash'
 'Slowpoke' 'Slowbro' 'Mega Slowbro' 'M

In [6]:
# compte les valeurs unique de legendaire
# liste_legendaires=df_legendaire.apply(pd.Series.value_counts)

In [7]:
# liste_legendaire = df[df['Legendary']==True].values
# print(liste_legendaire)

In [8]:
df.select_dtypes(include=[np.number]).info()
df.select_dtypes(include=['float64', 'int64']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   #           800 non-null    int64
 1   HP          800 non-null    int64
 2   Attack      800 non-null    int64
 3   Defense     800 non-null    int64
 4   Sp. Atk     800 non-null    int64
 5   Sp. Def     800 non-null    int64
 6   Speed       800 non-null    int64
 7   Generation  800 non-null    int64
dtypes: int64(8)
memory usage: 50.1 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   #           800 non-null    int64
 1   HP          800 non-null    int64
 2   Attack      800 non-null    int64
 3   Defense     800 non-null    int64
 4   Sp. Atk     800 non-null    int64
 5   Sp. Def     800 non-null    int64
 6   Speed       800 non-null    int64
 7   Generat

In [9]:
# Supprimer les colonnes avec des valeurs manquantes
df = df.dropna(axis=1)

# Sélectionner uniquement les colonnes numériques pour l'entraînement
X = df.select_dtypes(include=[np.number])
# pas besoin de la colonne index
X.drop(['#'], axis=1)
# Entraîner l'algorithme KNN
knn = NearestNeighbors(n_neighbors= range(1,10))
knn.fit(X)

InvalidParameterError: The 'n_neighbors' parameter of NearestNeighbors must be an int in the range [1, inf) or None. Got range(1, 10) instead.

In [ ]:
knn

NearestNeighbors(n_neighbors=range(1, 10))

In [ ]:
X.drop(['#'],axis=1)

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
0,45,49,49,65,65,45,1
1,60,62,63,80,80,60,1
2,80,82,83,100,100,80,1
3,80,100,123,122,120,80,1
4,39,52,43,60,50,65,1
...,...,...,...,...,...,...,...
795,50,100,150,100,150,50,6
796,50,160,110,160,110,110,6
797,80,110,60,150,130,70,6
798,80,160,60,170,130,80,6


In [ ]:
NearestNeighbors.kneighbors(X)

TypeError:        #  HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  Generation
0      1  45      49       49       65       65     45           1
1      2  60      62       63       80       80     60           1
2      3  80      82       83      100      100     80           1
3      4  80     100      123      122      120     80           1
4      5  39      52       43       60       50     65           1
..   ...  ..     ...      ...      ...      ...    ...         ...
795  796  50     100      150      100      150     50           6
796  797  50     160      110      160      110    110           6
797  798  80     110       60      150      130     70           6
798  799  80     160       60      170      130     80           6
799  800  80     110      120      130       90     70           6

[800 rows x 8 columns] is not an estimator instance.

In [ ]:


# Liste des Pokémon légendaires ou non que le champion utilise habituellement
champion_pokemons = ['Mewtwo', 'Lugia',
                     'Rayquaza', 'Giratina', 'Dialga', 'Palkia']


# Liste des Pokemon  legendaires
df_legendaire = df[df['Legendary'] == True]['Name'].unique()
# Liste des Pokemon non legendaires parmis lesquels le champion devra faire un choix
df_non_legendaire = df[df['Legendary'] == False]['Name'].unique()

# extraction des Pokemon legendaire de la liste des Pokemons preferés du champion
# Convertir df_legendaire_list et champion_pokemons en ensembles
df_legendaire_set = set(df_legendaire)
champion_pokemons_set = set(champion_pokemons)

# Obtenir les Pokémon qui sont dans df_legendaire_set et champion_pokemons_set
in_both = df_legendaire_set & champion_pokemons_set

# Obtenir les Pokémon qui sont dans champion_pokemons_set mais pas dans df_legendaire_set
not_in_df_legendaire = champion_pokemons_set - df_legendaire_set

print(f"Les Pokémons suivants sont legendaires : {list(in_both)}")
print(f"Les Pokémons preferés suivants pourront être utilisés par le champion : {
      list(not_in_df_legendaire)}")

# Modification  de la liste des pokemon que notre champion pourra utiliser  'Giratina'
champion_pokemons_ajusted = not_in_df_legendaire
print(champion_pokemons_ajusted)



In [ ]:

# Trouver le Pokémon non-légendaire le plus proche pour chaque Pokémon légendaire
for pokemon in df_non_legendaire:
    print("pokemon:", pokemon)
    pokemon_data = X[df['Name'] == pokemon]
    if pokemon_data.empty:
        print(f"{pokemon} n'est pas dans le DataFrame.")
    else:
        distance, index = knn.kneighbors(pokemon_data)
    print(f"pokemon_data:{pokemon_data}")
    distance, index = knn.kneighbors(pokemon_data)
    print(f"distance {distance}, index{index}")
    nearest_pokemon = df.iloc[index[0]]
    print(f"nearest_pokemon= {nearest_pokemon}")
    nearest_pokemon = nearest_pokemon[~nearest_pokemon['Legendary']]
    print(f"nearest_pokemon: {nearest_pokemon}")
    if not nearest_pokemon.empty:
        print(f"Le Pokémon non-légendaire le plus proche de {pokemon} est {
            nearest_pokemon['Name'].values[0]} avec une distance de {distance[0][0]}")
    else:
        print(
            f"Aucun Pokémon non-légendaire n'a été trouvé comme le plus proche de {pokemon}")

pokemon: Bulbasaur
pokemon_data:   #  HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  Generation
0  1  45      49       49       65       65     45           1
distance [[0.]], index[[0]]
nearest_pokemon=    #       Name Type 1  HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  \
0  1  Bulbasaur  Grass  45      49       49       65       65     45   

   Generation  Legendary  
0           1      False  
nearest_pokemon:    #       Name Type 1  HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  \
0  1  Bulbasaur  Grass  45      49       49       65       65     45   

   Generation  Legendary  
0           1      False  
Le Pokémon non-légendaire le plus proche de Bulbasaur est Bulbasaur avec une distance de 0.0
pokemon: Ivysaur
pokemon_data:   #  HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  Generation
1  2  60      62       63       80       80     60           1
distance [[0.]], index[[1]]
nearest_pokemon=    #     Name Type 1  HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  \
1  2  Ivysaur  Gra

In [ ]:
# X.drop(["#"], inplace=True)
# X.head(2)

In [13]:
distanceKNN = NearestNeighbors(n_neighbors=1).fit(X.reshape())
distanceKNN.kneighbors(X)

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [ ]:
# L’erreur ValueError: n_splits=5 cannot be greater than the number of members in each class. se produit lorsque le nombre de subdivisions (n_splits) dans une validation croisée est supérieur au nombre d’échantillons disponibles dans chaque classe12.

# Dans votre cas, vous utilisez GridSearchCV avec KNeighborsClassifier sur le jeu de données Pokémon. La variable cible y est le nom du Pokémon. Chaque Pokémon est unique, donc chaque classe (c’est-à-dire chaque Pokémon) n’a qu’un seul membre. Par défaut, GridSearchCV utilise une validation croisée stratifiée à 5 subdivisions (StratifiedKFold avec n_splits=5), ce qui signifie qu’elle essaie de diviser chaque classe en 5 subdivisions. Comme chaque classe n’a qu’un seul membre, vous obtenez cette erreur12.

# Pour résoudre ce problème, vous pouvez essayer de définir cv à KFold(n_splits=2) lors de l’initialisation de GridSearchCV, ce qui effectuera une validation croisée non stratifiée avec 2 subdivisions.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

y = df['Name']

grid = GridSearchCV(estimator=KNeighborsClassifier(),
                    param_grid={
                        "n_neighbors": range(2, 10),
                        "weights": ["uniform", "distance"],
                        "metric": ["minkowski", "manhattan", "euclidean"]},
                    cv=KFold(n_splits=3))

grid.fit(X,y)

GridSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
             estimator=KNeighborsClassifier(),
             param_grid={'metric': ['minkowski', 'manhattan', 'euclidean'],
                         'n_neighbors': range(2, 10),
                         'weights': ['uniform', 'distance']})

In [ ]:
grid.best_params_

{'metric': 'minkowski', 'n_neighbors': 2, 'weights': 'uniform'}

In [ ]:
df.columns

Index(['#', 'Name', 'Type 1', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def',
       'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [18]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Importer le dataset
df = pd.read_csv(
    'https://raw.githubusercontent.com/murpi/wilddata/master/pokemon.csv', sep=",")

# Supprimer les colonnes avec des valeurs nulles
df = df.dropna(axis=1)
print(df.shape)

# Sélectionner uniquement les colonnes numériques pour l'entraînement
X = df.select_dtypes(include=['float64', 'int64'])

# Entraîner l'algorithme k-NN pour trouver les 3 voisins les plus proches
k=3
# k=range(1,5)
knn = NearestNeighbors(n_neighbors=k)
# knn = NearestNeighbors(n_neighbors=3)
knn.fit(X)
champion_pokemons = ['Mewtwo', 'Lugia',
                     'Rayquaza', 'Giratina', 'Dialga', 'Palkia']
# Trouver les 3 Pokémon non-légendaires les plus proches pour chaque Pokémon légendaire du champion
recommendations = []
for pokemon in champion_pokemons:
    # if pokemon in df['Name'].values:
    if pokemon in df_legendaire:
        print(f"pokemon: {pokemon}")
        legendary_features = X[df['Name'] == pokemon]
        # legendary_features = X[df['Name'] == pokemon]
        print(f"legendary_features: {legendary_features}")
        distances, indices = knn.kneighbors(legendary_features)
        print(f"distance :{distances} \nindice: {indices}")
        for i in range(k):
            pokemon_plus_proche = df.iloc[indices[0][i]]
            print(f" i :{i} ")
            print(f"pokemon_plus_proche: {pokemon_plus_proche}")
            # Assurez-vous que le Pokémon recommandé n'est pas légendaire
            if not pokemon_plus_proche['Legendary']:
                recommendations.append(pokemon_plus_proche['Name'])
    else:
        print(f"Le Pokémon {pokemon} n'existe pas dans l'ensemble de données.")
print(set(recommendations))

(800, 11)


NameError: name 'df_legendaire' is not defined